`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

# 예제 05.001

## 문제에서 주어진 변수

### 각 구간 길이

In [ ]:
L_AB_m = 0.8
L_AD_m = 2.0
L_DE_m = 1.0

L_BD_m = 2.0 - L_AB_m
L_AE_m = L_AD_m + L_DE_m

### 하중

In [ ]:
w_AE_N_m = -6e3
P_D_N = -15e3

### 단면

In [ ]:
b_m = 120e-3
h_m = 200e-3
h_half_m = 0.5 * h_m

## a) 최대 굽힘 응력

최대 굽힘 모멘트가 발생하는 위치와 그 크기는 BMD 를 그려 보면 알 수 있다.

### A, B, C, D 지점의 x 좌표
A 점에서 x=0 으로 한다.

In [ ]:
x_A_m = 0
x_B_m = x_A_m + L_AB_m
x_D_m = x_B_m + L_BD_m
x_E_m = x_D_m + L_DE_m

### 반력

A점 중심의 모멘트는 분포하중, D점 하중, E점 반력이 각각 가하는 모멘트의 합이 된다.

$$
\int_{x=0}^{x=L_{AE}} wx \, dx+P_D L_{AD}+R_E L_{AE}=0\\
$$

E점 반력 $R_E$에 관련된 항만 $=$의 왼쪽에 남기고 나머지 항을 모두 오른쪽으로 옮긴다.

$$
R_E L_{AE}=-w\int_{x=0}^{x=L_{AE}} x \, dx-P_D L_{AD}\\
$$

$=$의 왼쪽과 오른쪽 모두를 $L_{AE}$ 로 나눈다.

$$
R_E=-\frac{w}{L_{AE}}\int_{x=0}^{x=L_{AE}} x \, dx-\frac{L_{AD}}{L_{AE}}P_D\\
$$

적분한다.

$$
R_E=-\frac{w}{L_{AE}}\left[\frac{1}{2}x^2\right]_{x=0}^{x=L_{AE}} 
-\frac{L_{AD}}{L_{AE}}P_D\\
$$

적분 구간을 대입한다.

$$
R_E=-\frac{wL_{AE}^2}{2L_{AE}}-\frac{L_{AD}}{L_{AE}}P_D\\
$$

분자와 분모의 $L_{AE}$를 약분한다.

$$
R_E=-\frac{wL_{AE}}{2}-\frac{L_{AD}}{L_{AE}}P_D\\
$$

E 점에서의 반력

In [ ]:
R_E_N = -0.5 * w_AE_N_m * L_AE_m - L_AD_m / L_AE_m * P_D_N
R_E_N

A 점에서의 반력
$$
R_A + w L_{AE} + P_D + R_E = 0 \\
R_A = -w L_{AE} - P_D - R_E
$$

In [ ]:
R_A_N = - w_AE_N_m * L_AE_m - P_D_N - R_E_N
R_A_N

### A-D 구간

In [ ]:
x_AD_m_array = np.linspace(x_A_m, x_D_m, 200+1)

#### 전단력선도

A 지점에서의 반력과 분포하중을 반영한다.

$$V_{AD}=R_A+wx$$

In [ ]:
V_AD_N_array = R_A_N + w_AE_N_m * x_AD_m_array

In [ ]:
plt.fill_between(x_AD_m_array, V_AD_N_array)
plt.ylabel('V(N)')

A 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax = plt.axes()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlim(xmax=x_E_m)
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

#### 굽힘모멘트 선도

A지점의 반력과 분포하중이 A-D 구간에 작용하는 모멘트를 반영한다.

$$M_{AD}=R_Ax+\frac{1}{2}wx^2$$

In [ ]:
M_AD_Nm_array = R_A_N * x_AD_m_array + 0.5 * w_AE_N_m * x_AD_m_array**2

In [ ]:
plt.fill_between(x_AD_m_array, M_AD_Nm_array)
plt.ylabel('M(Nm)')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlim(xmax=x_E_m)
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

### D-E 구간

In [ ]:
x_DE_m_array = np.linspace(x_D_m, x_E_m, 50+1)

#### 전단력선도

A, D 지점에서의 하중, 반력과 분포 하중을 감안하면 다음과 같다.

$$ V_{DE}=R_A+P_D+w  x $$

In [ ]:
V_DE_N_array = (R_A_N + P_D_N) + w_AE_N_m * x_DE_m_array

In [ ]:
plt.fill_between(x_AD_m_array, V_AD_N_array)
plt.fill_between(x_DE_m_array, V_DE_N_array)
plt.ylabel('V(N)')

A 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax = plt.axes()
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

D 지점에서의 하중을 화살표로 표시한다.

In [ ]:
ax.arrow(x_D_m, V_AD_N_array[-1], 
         0, P_D_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(P_D_N), fc='k', ec='k')

E 지점에서의 반력을 화살표로 표시한다.

In [ ]:
ax.arrow(x_E_m, V_DE_N_array[-1], 
         0, R_E_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_E_N), fc='k', ec='k')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

#### 굽힘모멘트 선도

전단력 선도를 적분한다.

In [ ]:
M_DE_Nm_array = si.cumtrapz(V_DE_N_array, x_DE_m_array, initial=0) + M_AD_Nm_array[-1]

In [ ]:
plt.fill_between(x_AD_m_array, M_AD_Nm_array)
plt.fill_between(x_DE_m_array, M_DE_Nm_array)
plt.ylabel('M(Nm)')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

### A-E 구간 전체

#### 전단력선도

In [ ]:
ax = plt.subplot(2, 1, 1)

In [ ]:
plt.fill_between(x_AD_m_array, V_AD_N_array)
plt.fill_between(x_DE_m_array, V_DE_N_array)
plt.ylabel('V(N)')

In [ ]:
ax.arrow(x_A_m, 0, 
         0, R_A_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_A_N), fc='k', ec='k')

In [ ]:
ax.arrow(x_D_m, V_AD_N_array[-1], 
         0, P_D_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(P_D_N), fc='k', ec='k')

In [ ]:
ax.arrow(x_E_m, V_DE_N_array[-1], 
         0, R_E_N * 0.9, 
         head_width=0.05, head_length=0.1 * abs(R_E_N), fc='k', ec='k')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)

#### 굽힘모멘트선도

In [ ]:
plt.subplot(2, 1, 2)

In [ ]:
plt.fill_between(x_AD_m_array, M_AD_Nm_array)
plt.fill_between(x_DE_m_array, M_DE_Nm_array)
plt.ylabel('M(Nm)')

In [ ]:
plt.text(x_A_m, 0, 'A')
plt.text(x_B_m, 0, 'B')
plt.text(x_D_m, 0, 'D')
plt.text(x_E_m, 0, 'E')
plt.xlabel('x(m)')
plt.grid(True)
plt.show()

D점에서 굽힘 모멘트가 최대가 된다.

A-D 구간의 마지막 모멘트 값은 다음과 같이 알 수 있다.

In [ ]:
M_AD_Nm_array[-1]

또는 다음과 같이 모멘트의 최대값을 찾도록 시킬 수 있다.

In [ ]:
max(M_AD_Nm_array)

In [ ]:
max(M_DE_Nm_array)

In [ ]:
M_max_Nm = max(max(M_AD_Nm_array), max(M_DE_Nm_array))
M_max_Nm

최대값의 $x$ 좌표는 다음과 같이 알 수 있다.

In [ ]:
index_max_M = np.argmax(M_AD_Nm_array)
x_AD_m_array[index_max_M]

굽힘 응력과 굽힘 모멘트 사이에는 다음과 같은 관계가 있다. (p. 170, eq 5.4a)

$$\sigma_{max}=\frac{|M|_{max}c}{I}=\frac{|M|_{max}}{\frac{I}{c}}=\frac{|M|_{max}}{S}$$

단면의 모멘트 $I$, $c$ 는 다음과 같다.

$$I=\frac{bh^3}{12}\\
c=\frac{h}{2}
$$

In [ ]:
I_m4 = b_m * h_m**3 / 12
c_m = h_m * 0.5

In [ ]:
sigma_max_Pa = M_max_Nm * c_m / I_m4
sigma_max_Pa

In [ ]:
sigma_max_MPa = sigma_max_Pa * 1e-6
sigma_max_MPa